In [1]:
import json
import random
import pandas as pd
import pickle
triples=pickle.load(open('triples_bison.p','rb'))+pickle.load(open('triples_bison2.p','rb'))
print(len(triples))

dfg1={'node_1':[],'node_2':[],'edge':[],'count':[],'chunk_id':[]}
for tri in triples[:]:
    try:
        tri=tri[1]
        tri=tri[8:-3]
        # print(tri)
        d=json.loads(tri)['triples']
        for t in d:
            if ('subject' not in t) or ('object' not in t) or ('predicate' not in t):
                continue
            dfg1['node_1'].append(t['subject'])
            dfg1['node_2'].append(t['object'])
            dfg1['edge'].append(t['predicate'])
            dfg1['chunk_id'].append(str(random.randint(0,1000000)))
            dfg1['count'].append(4)
    except KeyboardInterrupt:
        break
    except:
        continue
    
dfg1=pd.DataFrame(dfg1)
dfg1.head()

3802


,node_1,node_2,edge,count,chunk_id
0,cliente,abertura de conta,ter problema,4,492293
1,abertura de conta,em análise,estar,4,524326
2,abertura de conta,3 meses,ter tempo,4,506871
3,atendente,print,pedir,4,362641
4,atendente,solicitação interna,abrir,4,563305


In [2]:
dfg1.shape

(25205, 5)

In [3]:
#remove nan
dfg1=dfg1.dropna()

## Filtragens

### Global

In [4]:
#remover dos clientes e atendentes as triplas com frequencia menor que freq
freq_triplas={}
for i in range(len(dfg1)):
    tripla=dfg1.iloc[i]
    tripla_nodes=tripla['node_1']+' '+tripla['node_2']
    if tripla_nodes not in freq_triplas:
        freq_triplas[tripla_nodes]=0
    freq_triplas[tripla_nodes]+=1

In [5]:
#remover os que tem frequencia menor que freq
freq=3
dfg1['tripla_nodes']=dfg1['node_1']+' '+dfg1['node_2']
dfg1['freq']=dfg1['tripla_nodes'].apply(lambda x: freq_triplas[x])
dfg1=dfg1[dfg1['freq']>=freq]
dfg1.shape

(8983, 7)

In [6]:
dfg1

,node_1,node_2,edge,count,chunk_id,tripla_nodes,freq
0,cliente,abertura de conta,ter problema,4,492293,cliente abertura de conta,15
3,atendente,print,pedir,4,362641,atendente print,61
4,atendente,solicitação interna,abrir,4,563305,atendente solicitação interna,3
6,cliente,e-mail,ter retorno,4,796684,cliente e-mail,75
7,fatura,não paga,consta,4,805653,fatura não paga,3
...,...,...,...,...,...,...,...
25182,atendente,autenticação,enviar,4,992751,atendente autenticação,10
25190,atendente,confirmação de dados,pede,4,494353,atendente confirmação de dados,8
25191,atendente,notificação,envia,4,286391,atendente notificação,22
25192,cliente,notificação,aceita,4,101255,cliente notificação,34


### Cliente e atendente

In [7]:
#processamento: pegar todas as triplas clientes e atendentes
clientes_triplas=dfg1[dfg1['node_1'].str.contains('cliente')]
atendentes_triplas=dfg1[dfg1['node_1'].str.contains('atendente')]
#remover triplas de dfg1
dfg1=dfg1[~dfg1['node_1'].str.contains('cliente')]
dfg1=dfg1[~dfg1['node_1'].str.contains('atendente')]

clientes_triplas.shape, atendentes_triplas.shape, dfg1.shape

((5542, 7), (1605, 7), (1836, 7))

In [8]:
#remover dos clientes e atendentes as triplas com frequencia menor que freq
freq_cliente_atendente_triplas={}
for i in range(len(clientes_triplas)):
    tripla=clientes_triplas.iloc[i]
    tripla_nodes=tripla['node_1']+' '+tripla['node_2']
    if tripla_nodes not in freq_cliente_atendente_triplas:
        freq_cliente_atendente_triplas[tripla_nodes]=0
    freq_cliente_atendente_triplas[tripla_nodes]+=1
for i in range(len(atendentes_triplas)):
    tripla=atendentes_triplas.iloc[i]
    tripla_nodes=tripla['node_1']+' '+tripla['node_2']
    if tripla_nodes not in freq_cliente_atendente_triplas:
        freq_cliente_atendente_triplas[tripla_nodes]=0
    freq_cliente_atendente_triplas[tripla_nodes]+=1

In [9]:
#remove triplas com frequencia menor que freq
freq=15
clientes_triplas=clientes_triplas[clientes_triplas.apply(lambda x: freq_cliente_atendente_triplas[x['node_1']+' '+x['node_2']]>=freq,axis=1)]
atendentes_triplas=atendentes_triplas[atendentes_triplas.apply(lambda x: freq_cliente_atendente_triplas[x['node_1']+' '+x['node_2']]>=freq,axis=1)]
clientes_triplas.shape, atendentes_triplas.shape

((3048, 7), (836, 7))

In [10]:
clientes_triplas.head()

,node_1,node_2,edge,count,chunk_id,tripla_nodes,freq
0,cliente,abertura de conta,ter problema,4,492293,cliente abertura de conta,15
6,cliente,e-mail,ter retorno,4,796684,cliente e-mail,75
17,cliente,cartão bloqueado,tem problema,4,527327,cliente cartão bloqueado,133
22,cliente,pix,tenta fazer,4,94049,cliente pix,81
28,cliente,atendente,agradece,4,967281,cliente atendente,225


In [11]:
#adicionar triplas de clientes e atendentes a dfg1
dfg1=pd.concat([dfg1,clientes_triplas,atendentes_triplas])  
dfg1.shape

(5720, 7)

## Visualizacao

In [12]:
import numpy as np
import os

In [13]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
807,via e-mail,retorno,"335267,823065,618209,847985",4,contextual proximity
808,vinculada,conta investimento,"698081,213268,291584,233719,73079",5,contextual proximity
809,virtual,cartão,"859883,466443,703481,390178,40940,334424,566259",7,contextual proximity
810,vitreo,conta,"272066,936815,544596,658216",4,contextual proximity
811,zerada,conta,"971435,930968,2474",3,contextual proximity


In [14]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,11/02,fatura,"515844,965073,883285",contextual proximity,3
1,12 meses,isenção,"836590,132270,942515,327698",contextual proximity,4
2,14/02,pagamento,"530239,655186,652236",contextual proximity,3
3,15 dias,estabelecimento,"130073,28003,515903,64436,408949",contextual proximity,5
4,15 dias,estorno,"429527,452069,40624,485614",contextual proximity,4
...,...,...,...,...,...
760,via e-mail,retorno,"335267,823065,618209,847985",contextual proximity,4
761,vinculada,conta investimento,"698081,213268,291584,233719,73079",contextual proximity,5
762,virtual,cartão,"859883,466443,703481,390178,40940,334424,566259",contextual proximity,7
763,vitreo,conta,"272066,936815,544596,658216",contextual proximity,4


Calculate the NetworkX Graph


In [15]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(286,)

In [16]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [17]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
# print(communities)

Number of Communities =  9


In [18]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,11/02,#db5f57,1
1,14/02,#db5f57,1
2,15 dias,#db5f57,1
3,24h e 7 dias por semana,#db5f57,1
4,29/02/24,#db5f57,1
...,...,...,...
281,mercado,#5f57db,7
282,funcionalidade de segurança,#57a7db,8
283,token,#57a7db,8
284,não determinado,#dbb757,9


In [19]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [20]:
from pyvis.network import Network

graph_output_directory = "/home/kiki/Projects/BTG/graph/data_output/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

/home/kiki/Projects/BTG/graph/data_output/index.html
